# Analysing german news articles

## Imports

In [2]:
# Pandas is an open source data analysis and manipulation tool
import pandas as pd
# os gives access to the operating system
import os
# The datetime module supplies classes for manipulating dates and times.
from datetime import datetime 
import datetime
# This module provides various time-related functions.
import time

# Natural language toolkit
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# library to create visualizations
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.offsetbox import AnchoredText


# open source library for automating downloading of reports from Google Trends
from pytrends.request import TrendReq


## Load data into DataFrames

In [38]:
# Folderpath to CSV's
CSV_folderPath="/Users/jan/Documents/Python_Projects/Bachelorthesis/CSV_Backlog"

# Initialize list<csv> for Spiegel, Sueddeutsche, Bild and combined 
Spiegel_csvs = []
Sueddeutsche_csvs = []
Bild_csvs = []
Combi_csvs = []


# Change directory to CSV_folderPath
os.chdir(CSV_folderPath)

# iterate over all files
for file in os.listdir():
    # that are csvs...
    if file.endswith(".csv"):        
        # read csv to dataframe
        df =pd.read_csv(file, index_col=None, header=0)
        # all df append to combi...
        Combi_csvs.append(df)
        # sort dataframe to its list
        #Spiegel
        if file.startswith("Spiegel"):
            Spiegel_csvs.append(df)
        # Sueddeutsche
        elif file.startswith("Sueddeutsche"):
            Sueddeutsche_csvs.append(df)
        # Bild
        elif file.startswith("Bild"):
            Bild_csvs.append(df)
    
All_articles = pd.concat(Combi_csvs, axis=0, ignore_index=True)
Spiegel_articles = pd.concat(Spiegel_csvs, axis=0, ignore_index=True)
Sueddeutsche_articles = pd.concat(Sueddeutsche_csvs, axis=0, ignore_index=True)
if(len(Bild_csvs)) != 0:
    Bild_articles = pd.concat(Bild_csvs, axis=0, ignore_index=True)
else:
    Bild_articles=[]

    
# Drop duplicates

# All articles
All_articles= All_articles.drop_duplicates(subset="Titel")

# Spiegel articles
Spiegel_articles = Spiegel_articles.drop_duplicates(subset="Titel")

# Sueddeutsch articles
Sueddeutsche_articles = Sueddeutsche_articles.drop_duplicates(subset="Titel")

# Bild articles
#Bild_articles = Bild_articles.drop_duplicates(subset="title")
All_articles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29436 entries, 0 to 70452
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Titel                  29436 non-null  object 
 1   Date_Info              29436 non-null  object 
 2   URL                    29436 non-null  object 
 3   Overline               20256 non-null  object 
 4   Breadcrumb             14851 non-null  object 
 5   author                 3643 non-null   object 
 6   detailed_informations  5148 non-null   object 
 7   Zugriff_Datum          20872 non-null  float64
 8   News_page              29436 non-null  object 
dtypes: float64(1), object(8)
memory usage: 2.2+ MB


## Data Cleaning

### Cast Date_Info to datetime

In [39]:
# All articles
All_articles["Date_Info"] = All_articles["Date_Info"].str.strip()
All_articles["Date_Info"] = pd.to_datetime(All_articles["Date_Info"], format='%d%m%Y_%H:%M')
# Spiegel articles
Spiegel_articles["Date_Info"] = Spiegel_articles["Date_Info"].str.strip()
Spiegel_articles["Date_Info"] = pd.to_datetime(Spiegel_articles["Date_Info"], format='%d%m%Y_%H:%M')
# Sueddeutsche articles
Sueddeutsche_articles["Date_Info"] = Sueddeutsche_articles["Date_Info"].str.strip()
Sueddeutsche_articles["Date_Info"] = pd.to_datetime(Sueddeutsche_articles["Date_Info"], format='%d%m%Y_%H:%M')
# Bild articles
#Bild_carticles["Date_Info"] = pd.to_datetime(Bild_carticles["Date_Info"], format='%d%m%Y_%H:%M')

## Initial data analysis

### Ammount of articles

In [40]:
# Spiegel articles
ammount_of_Spiegel_articles = len(Spiegel_articles)
print(f"Spiegel articles: {ammount_of_Spiegel_articles}")

# Sueddeutsche articles
ammount_of_Sueddeutsche_articles = len(Sueddeutsche_articles)
print(f"Sueddeutsche articles: {ammount_of_Sueddeutsche_articles}")

# Bild articles
ammount_of_Bild_articles = len(Bild_articles)
print(f"Bild articles: {ammount_of_Bild_articles}")

# All articles
ammount_of_Combined_articles = len(All_articles)
print(f"Overall: {ammount_of_Combined_articles}")

Spiegel articles: 8564
Sueddeutsche articles: 20873
Bild articles: 0
Overall: 29436


# Helper

### Exploding titles

In [41]:
def explode_titles(title_column):
    title_column =title_column.to_frame()
    tokenized_titles=[]
    title_column["tokenized_title"] = ""
    for index, row in title_column.iterrows():
        # tokenize
        tokenized_title = word_tokenize(row.Titel)
        
        # remove stopwords & numbers/punction
        tokenized_title = [word for word in tokenized_title if word not in stopwords.words("german")]
        
        # add to row
        row.tokenized_title = tokenized_title
        
    return title_column

### Google

In [112]:
def get_google_interest_over_time(keyword_list):
    # connect to google
    pytrends = TrendReq(hl='de', tz='60') 

    # keywords
    keywords = keyword_list

    # build payload
    startDate = "2021-11-07"
    dateTime=datetime.datetime.now()
    currentDate = dateTime.strftime("%Y-%m-%d")
    timeframe = startDate +" "+ str(currentDate)
    dataframes = []
    for keyword in keywords:
        keyword_list = []
        keyword_list.append(keyword)
        pytrends.build_payload(keyword_list, cat=0, timeframe= str(timeframe) ) 
        
        # get data with interest over time
        data = pytrends.interest_over_time() 
        dataframes.append(data)
        time.sleep(2)
    
    # concat all google trends results
    dfs = pd.concat(dataframes, axis=1)
    # remove deprecated column
    dfs=dfs.loc[:,~dfs.columns.str.startswith('isPartial')]

    # get df to right format
    google_df = dfs.unstack().reset_index() 
    google_df = google_df.rename(columns={"level_0" : "KeyWord", 0 : "Occurence_in_Google"})
    
    #fill all NaN with 0
    google_df["Occurence_in_Google"] = google_df["Occurence_in_Google"].fillna(0)
    google_df["Occurence_in_Google"] = google_df["Occurence_in_Google"].astype(int)
    
    google_df["Occurence_in_Google_normalized"] = google_df["Occurence_in_Google"].div(100)
    return google_df

### Titles 

In [45]:
def get_occurence_of_all_titles(data_frame, columnName):
    # Group titles by columnName
    all_titles = data_frame.groupby(columnName).size()
    
    #sort titles
    all_titles = all_titles.sort_values(ascending = False)
    
    return all_titles

def get_occurence_of_all_capital_titles(data_frame, columnName):
    all_titles=data_frame.groupby(columnName).size()

    capital_titles=[]
    counter=0
    for title in all_titles.items():
        title_s=str(title)
        title_s=title_s.strip()
        if title_s.istitle():
            capital_titles.append(title)        
    Titles = pd.DataFrame.from_records(
    capital_titles, columns=['Title','Occurence'])
    return Titles.sort_values(by=['Occurence'],ascending=False)

def get_titles_with_minimum_occurence_N(data_frame, N):
    all_Titles = get_occurence_of_all_capital_titles(data_frame, "tokenized_titles")
    above_N = []
    for index,row in all_Titles.iterrows():
        if int(row.Occurence) >= N:
            above_N.append(row)
            
    above_N = pd.DataFrame(above_N, columns=['Title', 'Occurence'])

    return data_frame[data_frame["tokenized_titles"].isin(above_N.Title)]

def get_titles_with_minimum_occurence_N(data_frame, N):
    all_Titles = get_occurence_of_all_capital_titles(data_frame, "tokenized_titles")
    above_N = []
    for index,row in all_Titles.iterrows():
        if int(row.Occurence) >= N:
            above_N.append(row)
            
    above_N = pd.DataFrame(above_N, columns=['Title', 'Occurence'])

    return data_frame[data_frame["tokenized_titles"].isin(above_N.Title)]        

# Build big Dataframe

In [42]:
All_articles["tokenized_titles"] = explode_titles(All_articles["Titel"]).tokenized_title
All_articles = All_articles.explode("tokenized_titles")
uninteresting_titles= ["Der", "Die", "Das","Was","Warum","Er","Sie", "Es", "Ich", "Du", "Mit","Wie","Ein","So","Wir",
                       ":","»","«", "," ,"\'\'","``", "_","-" ".","?","–", "-",".","!"]

All_articles=All_articles[~All_articles.tokenized_titles.isin(uninteresting_titles)]


## Analysis

### Ammount of articles with occurence above n

In [13]:
occurence = [10,25,50,75,100,125,150,200]
for n in occurence:
    x = get_titles_with_minimum_occurence_N(All_articles, n)
    y = get_occurence_of_all_capital_titles(x,"tokenized_titles")
    print("n:"+str(n) +" = "+str(len(y)))

n:10 = 1811
n:25 = 584
n:50 = 239
n:75 = 125
n:100 = 72
n:125 = 48
n:150 = 35
n:200 = 21


# Build Occurence DF

In [114]:
def build_Occurence_df(dataframe, n_occurences):
    article_over_occurence = get_titles_with_minimum_occurence_N(dataframe[["Date_Info", "tokenized_titles"]], n_occurences)

    # prep keyword list "Titles"
    keyword_list = list(article_over_occurence["tokenized_titles"].drop_duplicates())

    google_data = get_google_interest_over_time(keyword_list)

    #rename columns
    article_over_occurence = article_over_occurence.rename(columns={"Date_Info" : "date", "tokenized_titles" : "Occurence_in_news"})
    #convert to datetime
    article_over_occurence['date'] = pd.to_datetime(article_over_occurence.date) 
    #convert datetime format to googles datetime format
    article_over_occurence['date'] = article_over_occurence['date'].dt.strftime('%Y-%m-%d')
    #reshape our data frame to look like google data frame
    occurence_df=article_over_occurence.pivot_table(index='date', columns='Occurence_in_news', aggfunc='size') \
                .rename_axis(None, axis=1)
    #fill all NaN with 0
    occurence_df = occurence_df.fillna(0)
    occurence_df = occurence_df.astype(int)

    # bring news df to right format
    news_df = occurence_df.unstack().reset_index()
    news_df = news_df.rename(columns={"level_0" : "KeyWord", 0 : "Occurence_in_News"})
    news_df["date"]=pd.to_datetime(news_df["date"], format='%Y-%m-%d')
    
    # normalize Occurence_in_News
    
    # merge google and news
    full_df = pd.merge(news_df, google_data, on=['KeyWord',"date"], how='outer')
    #fill all NaN with 0
    full_df["Occurence_in_Google"] = full_df["Occurence_in_Google"].fillna(0)
    full_df["Occurence_in_Google"] = full_df["Occurence_in_Google"].astype(int)
    data_frame_list=[]
    for keyword in keyword_list:
        keyworded_dataFrame = full_df.loc[full_df["KeyWord"] == keyword]

        df_chunk = keyworded_dataFrame
        df_chunk["Occurence_in_News_Normalized"]=(keyworded_dataFrame["Occurence_in_News"]-keyworded_dataFrame["Occurence_in_News"].min())/(keyworded_dataFrame["Occurence_in_News"].max()-keyworded_dataFrame["Occurence_in_News"].min())
    
        data_frame_list.append(df_chunk)
    merged = pd.concat(data_frame_list)
    return merged
    
    

In [ ]:
build_Occurence_df(All_articles,50)



In [111]:
merged

,KeyWord,date,Occurence_in_News,Occurence_in_Google,Occurence_in_News_Normalized
790,Mann,2021-11-07,1,0.94,0.047619
791,Mann,2021-11-08,5,0.88,0.238095
792,Mann,2021-11-09,2,0.83,0.095238
793,Mann,2021-11-10,0,0.81,0.000000
794,Mann,2021-11-11,1,0.80,0.047619
...,...,...,...,...,...
1259,Neue,2022-01-20,3,0.29,0.333333
1260,Neue,2022-01-21,8,0.25,0.888889
1261,Neue,2022-01-22,1,NaN,0.111111
1262,Neue,2022-01-23,5,NaN,0.555556


In [93]:
full_df

,KeyWord,date,Occurence_in_News,Occurence_in_Google
0,Auto,2021-11-07,0,0.81
1,Auto,2021-11-08,0,0.96
2,Auto,2021-11-09,1,0.95
3,Auto,2021-11-10,0,0.94
4,Auto,2021-11-11,0,0.92
...,...,...,...,...
1654,Zwei,2022-01-20,7,0.86
1655,Zwei,2022-01-21,7,0.88
1656,Zwei,2022-01-22,4,NaN
1657,Zwei,2022-01-23,6,NaN


In [ ]:
# NEWSPAGE
# get all titles that occure above N
article_over_occurence = get_titles_with_minimum_occurence_N(All_articles[["Date_Info", "tokenized_titles"]], 50)

# prep keyword list "Titles"
keyword_list = list(article_over_occurence["tokenized_titles"].drop_duplicates())

#rename columns
article_over_occurence = article_over_occurence.rename(columns={"Date_Info" : "date", "tokenized_titles" : "Occurence_in_news"})
#convert to datetime
article_over_occurence['date'] = pd.to_datetime(article_over_occurence.date) 
#convert datetime format to googles datetime format
article_over_occurence['date'] = article_over_occurence['date'].dt.strftime('%Y-%m-%d')
#reshape our data frame to look like google data frame
occurence_df=article_over_occurence.pivot_table(index='date', columns='Occurence_in_news', aggfunc='size') \
            .rename_axis(None, axis=1)
#fill all NaN with 0
occurence_df = occurence_df.fillna(0)
occurence_df = occurence_df.astype(int)

# GOOGLE 
# google trends laden
data = google_interest_over_time(keyword_list)
# concat all google trends results
dfs = pd.concat(data, axis=1)
# remove deprecated column
dfs=dfs.loc[:,~dfs.columns.str.startswith('isPartial')]


# get current datetime for later filenames
date_time = datetime.datetime.today().strftime("%d_%m_%Y")
# list where all plots land
all_plots=[]
# list where 
#iterate over keyword list
for keyword in keyword_list:
    # get series for news
    news_data = occurence_df[keyword]
    # get series for google
    google_data = dfs[keyword]
    
    # rename the Series
    news_data= news_data.rename("News_Data")
    google_data = google_data.rename("Google_Data")
    
    # cast to dataframe 
    news_data = news_data.to_frame()
    google_data = google_data.to_frame()
    # reset index
    news_data = news_data.reset_index()
    
    news_data['date'] = pd.to_datetime(news_data.date) 
    merged_data = pd.merge(news_data,google_data, how="outer", on="date")
    merged_data.to_csv("/Users/jan/Documents/Python_Projects/Bachelorthesis/Analysis/News_vs_Google/news_vs_google"+date_time+"_"+keyword +".csv",index=False)
    all_plots.append(createFigure(merged_data,keyword))


# save all figures in one pdf
with PdfPages("/Users/jan/Documents/Python_Projects/Bachelorthesis/Analysis/News_vs_Google/news_vs_google"+date_time +".pdf") as pdf:
    for plot in all_plots:
        pdf.savefig(plot)


In [ ]:
def createFigure(data_frame, fileName):
    plot = data_frame.plot.line(x="date",y=[1,2],figsize=(20,10), title="Occurence of "+fileName)
    Correlation = data_frame["News_Data"].corr(data_frame["Google_Data"])
    plot.annotate("Correlation: " + str(Correlation) , xy=(0.05, 0.95), xycoords='axes fraction')    
    return plot.get_figure()



## Plots

### Plot occurence of top 50 titles

In [ ]:
# get all titles
all_titles = get_occurence_of_all_titles(All_articles,"tokenized_titles")

# get top 50
top50_titles = all_titles[:50]

plt.figure(figsize=(20,3))
top50_titles.plot(kind="bar")

### Plot occurence of top 50 titles starting with capital letter

In [ ]:
# get all titles
all_titles = get_occurence_of_all_capital_titles(All_articles,"tokenized_titles")

#get top 50
top50_titles = all_titles[:50]

#plot
top50_titles.plot.bar(x="Title",y="Occurence",figsize=(20, 3))
